## Settings

In [1]:
import sys
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/")

#NumCosmo
from numcosmo_py import Ncm, Nc, GObject
from richness_mass_calib import create_richness_mass_calib

#Useful packages
import numpy as np
import math
from scipy import stats

from astropy.io import fits
from astropy.table import Table
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

## Data

In [2]:
# DC2 data
DC2_halos_m200c = fits.open(
    "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits")

dt_halos = Table(DC2_halos_m200c[1].data)

lnM = np.log(dt_halos["m200c"])  # ln(mass)
z = dt_halos["redshift_true"]  # redshift
lnR = np.log(dt_halos["richness"])  # ln(richness)



## Bins

Nestas células fazemos um recorte de intervalos de z de largura 0.05 e esses intervalos são colocados em **halos_bin_z**. Depois, pegamos cada *bin* em z e cortamos em intervalos de lnM de largura 0.25. Os bins são colocados em **halos_bin_mz**.

Para cada intervalo em **halos_bin_mz** é calculado lnR médio, z médio e lnM médio e as listas desses valores são colocadas na tabela **halos mean**.

In [3]:
# redshift bins
z_0 = 0.0
z_1 = 0.05
halos_bin_z =[]

for i in range(61):
    cut_z = np.logical_and (dt_halos['redshift_true'] > z_0, dt_halos['redshift_true'] < z_1)
    halos_bin_z.append(dt_halos[cut_z])
    z_0 = z_0 + 0.05
    z_1 = z_1 + 0.05

# mass bins
label = []
halos_bin_mz =[]
for i in range(61):
    
    lnM_0 = min(lnM)
    lnM_1 = min(lnM) + 0.25
    for j in range(21):
        
        cut = np.logical_and (np.log(halos_bin_z[i]["m200c"]) > lnM_0, np.log(halos_bin_z[i]["m200c"]) < lnM_1)
        halos_bin_mz.append(halos_bin_z[i][cut])
        label.append(f"{min(halos_bin_z[i]['redshift_true']):.3f} < z < {max(halos_bin_z[i]['redshift_true']):.3f}\n{lnM_0:.3f} < lnM < {lnM_1:.3f}")
        
        lnM_0 = lnM_0 + 0.25
        lnM_1 = lnM_1 + 0.25
        
len(halos_bin_mz)

1281

In [4]:
lnM_binned, z_binned, lnR_binned = [], [], []

for i in range(len(halos_bin_mz)):
    
    halos = halos_bin_mz[i]
    lnM_binned.append(np.log(halos["m200c"]))
    z_binned.append(halos["redshift_true"])
    lnR_binned.append(np.log(halos["richness"]))   

lnR_mean, lnM_mean, z_mean = [np.mean(l) for l in lnR_binned if len(l) > 0], [np.mean(l) for l in lnM_binned if len(l) > 0], [np.mean(k) for k in z_binned if len(k) > 0]

halos_mean = Table([np.exp(np.array(lnR_mean)), np.exp(np.array(lnM_mean)), z_mean],
           names=('richness', 'm200c', 'redshift_true'))

In [ ]:
fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

zs = lnR_mean
xs = z_mean
ys = lnM_mean

p3 =ax1.scatter(xs, ys, zs, c=zs, cmap='rainbow')
ax1.set_xlabel('z', fontsize = 14)
ax1.set_ylabel('lnM', fontsize = 14)
fig.colorbar(p3, ax=ax1).set_label(label='$ \mu_{\ln R} $',size=14)

p4 = ax2.scatter(ys, zs , c= xs, cmap='rainbow')
ax2.set_xlabel('lnM', fontsize = 14)
ax2.set_ylabel('$ \mu_{\ln R} $', fontsize = 14)

fig.colorbar(p4, ax=ax2).set_label(label='z',size=14)


p5 = ax3.scatter(xs, zs , c=ys, cmap='rainbow') 
ax3.set_xlabel('z', fontsize = 14)
ax3.set_ylabel('$ \mu_{\ln R} $',  fontsize = 14)
fig.colorbar(p5, ax=ax3).set_label(label='lnM',size=14)


fig.suptitle('$ \mu_{\ln R} $ em função de lnM e z ', size = 18, y=1.10)

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

lnz = np.log(np.array(z_mean))

lnzp = ax.scatter(lnz, zs , c=ys, cmap='rainbow') 
ax.set_xlabel('$\ln$z', fontsize = 14)
ax.set_ylabel('$ \mu_{\ln R} $',  fontsize = 14)
ax.set_title("$\mu_{\ln R} $ x $\ln$z ", fontsize=16)
fig.colorbar(lnzp, ax=ax).set_label(label='lnM',size=14)

plt.show()

In [5]:
def Model_fit(mod):
    dt_halos = Table(DC2_halos_m200c[1].data)
    rmdata = create_richness_mass_calib(dt_halos)
    fixed_parameters = []
    
    match mod:
        case "ext":
            model = Nc.ClusterMassLnrichExt()
            model.get_property('use_ln1pz')
            fixed_parameters = [12, 13, 14]
            
        case "ext2":
            model = Nc.ClusterMassLnrichExt()
            fixed_parameters = [12, 13, 14]
            
        case "ascaso":
            model = Nc.ClusterMassAscaso()
            fixed_parameters = [6]
        
    mset = Ncm.MSet()
    mset.set(model)
    rmdata.m2lnL_val(mset)  
    
    dset = Ncm.Dataset.new()
    dset.append_data(rmdata)
    lh = Ncm.Likelihood.new(dset)

    mset.param_set_all_ftype(Ncm.ParamType.FREE)
    for par in fixed_parameters:
        mset.param_set_ftype(7000, par, Ncm.ParamType.FIXED)
    mset.prepare_fparam_map()

    fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )
    fit.log_info()
    fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)
    fit.log_info()
    
    lnR_mean_model = np.array([model.get_mean_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])
    lnR_std_model = np.array( [model.get_std_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])
    
    return lnR_mean_model, lnR_std_model

Model_fit('ext')

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  3.19                [FREE]
#   -    muM1[01]:  0.868588963806504   [FREE]
#   -    muZ1[02]: -0.304006137332276   [FREE]
#   -    muM2[03]:  0                   [FREE]
#   -    muZ2[04]:  0                   [FREE]
#   -    muMZ[05]:  0                   [FREE]
#   -  sigma0[06]:  0.33                [FREE]
#   - sigmaM1[07]: -0.0347435585522601  [FREE]
#   - sigmaZ1[08]:  0                   [FREE]
#   - sigmaM2[09]:  0                   [FREE]
#   - sigmaZ2[10]:  0                   [FREE]
#   - sigmaMZ[11]:  0                   [FREE]
#   -     cut[12]:  6                   [FIX

(array([1.4164755 , 1.55878698, 1.66439255, 1.81973995, 1.95696378,
        2.09852121, 2.31959656, 2.46751198, 2.6420654 , 2.84100399,
        2.97419423, 3.2363835 , 3.26996243, 4.25503732, 1.45827139,
        1.60417556, 1.73862116, 1.87766389, 2.04060754, 2.18628146,
        2.33394595, 2.49243708, 2.67608277, 2.82057555, 2.95374505,
        3.1057433 , 3.33405383, 1.53210367, 1.66793466, 1.80047357,
        1.93832338, 2.09733745, 2.24907017, 2.40323569, 2.5574189 ,
        2.71043452, 2.90915125, 3.11191549, 3.22750519, 3.44482078,
        3.67565013, 3.81913737, 4.2427729 , 1.59540734, 1.72892718,
        1.86788302, 2.01518647, 2.16138727, 2.3234589 , 2.47232783,
        2.64738538, 2.8149768 , 2.99899989, 3.17442491, 3.36351233,
        3.57448691, 3.74846382, 3.94968276, 4.17752448, 4.30123878,
        4.65785637, 4.90017173, 1.64847435, 1.78308191, 1.92840713,
        2.06480846, 2.22125986, 2.37952967, 2.5497007 , 2.70136602,
        2.88672569, 3.05582963, 3.22774335, 3.43

## Ascaso Fit Params

Aqui fazemos o ajuste do modelo de ascaso e encontramos os valores dos parâmetros usando todos os dados em **dt_halos** (sem dividir em intervalos). 

\begin{equation}
    \left< \ln R | M, z \right> = \mu_0 + \mu_1 \ln\left( \frac{M}{M_0} \right) +  \mu_2\ln\left(\frac{1 + z}{1 + z_0}\right);
\end{equation}

$$\textbf{p}_{1} = \{\mu_0, \mu_1, \mu_2, \sigma_0, \sigma_1, \sigma_2\}. $$



In [ ]:
dt_halos = Table(DC2_halos_m200c[1].data)
rmdata = create_richness_mass_calib(dt_halos)
ascaso = Nc.ClusterMassAscaso()
ascaso.param_set_by_name("cut", 1e15)

mset = Ncm.MSet()
mset.set(ascaso)

rmdata.m2lnL_val(mset)

In [ ]:
dset = Ncm.Dataset.new()
dset.append_data(rmdata)

lh = Ncm.Likelihood.new(dset)

mset.param_set_all_ftype(Ncm.ParamType.FREE)
mset.param_set_ftype(7000, 6, Ncm.ParamType.FIXED)

mset.prepare_fparam_map()

fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )

In [ ]:
fit.log_info()

In [ ]:
fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)

In [ ]:
fit.log_info()

In [ ]:
lnR_mean_ascaso = np.array([ascaso.get_mean_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])
lnR_std_ascaso = np.array( [ascaso.get_std_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])

## Ext Fit Params

Aqui fazemos o ajuste do modelo extendido, também usando todos os dados em **dt_halos** .

\begin{equation}\label{M_R_3}
\begin{split}
    \left<\ln R | M, z \right> = \mu_0 + 
    \mu_{M1} \ln\left( \frac{M}{M_0} \right) +
    \mu_{M2} (\ln\left(\frac{M}{M_0} \right))^2 +\\
    + \mu_{z1} \ln\left(\frac{1 + z}{1 + z_0}\right) +
    \mu_{z2} (\ln\left(\frac{1 + z}{1 + z_0}\right))^2 +\\
    + \mu_{Mz} \ln\left(\frac{1 + z}{1 + z_0}\right)\ln\left(\frac{M}{M_0} \right);
\end{split}
\end{equation}

$$\textbf{p}_{2} = \{\mu_0, \mu_{M1}, \mu_{M2}, \mu_{Z1}, \mu_{Z2}, \mu_{MZ}, \sigma_0, \sigma_{M1}, \sigma_{M2}, \sigma_{Z1}, \sigma_{Z2}, \sigma_{MZ} \}.$$

In [ ]:
dt_halos = Table(DC2_halos_m200c[1].data)

lnM = np.log(dt_halos["m200c"])  # ln(mass)
z = dt_halos["redshift_true"]  # redshift
lnR = np.log(dt_halos["richness"])  # ln(richness)


rmdata = create_richness_mass_calib(dt_halos)

ext = Nc.ClusterMassLnrichExt()
ext.get_property('use_ln1pz')
ext.param_set_by_name("cut", 1e15)

mset = Ncm.MSet()
mset.set(ext)

rmdata.m2lnL_val(mset)

dset = Ncm.Dataset.new()
dset.append_data(rmdata)

lh = Ncm.Likelihood.new(dset)

mset.param_set_all_ftype(Ncm.ParamType.FREE)
mset.param_set_ftype(7000, 12, Ncm.ParamType.FIXED)
mset.param_set_ftype(7000, 13, Ncm.ParamType.FIXED)
mset.param_set_ftype(7000, 14, Ncm.ParamType.FIXED)


mset.prepare_fparam_map()

fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )

fit.log_info()

In [ ]:
fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)

In [ ]:
fit.log_info()

In [ ]:
lnR_mean_ext = np.array([ext.get_mean_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])
lnR_std_ext = np.array( [ext.get_std_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])

### EXT 2

In [ ]:
dt_halos = Table(DC2_halos_m200c[1].data)

lnM = np.log(dt_halos["m200c"])  # ln(mass)
z = dt_halos["redshift_true"]  # redshift
lnR = np.log(dt_halos["richness"])  # ln(richness)


rmdata = create_richness_mass_calib(dt_halos)

ext2 = Nc.ClusterMassLnrichExt()
ext2.param_set_by_name("cut", 1e15)

mset = Ncm.MSet()
mset.set(ext2)

rmdata.m2lnL_val(mset)

dset = Ncm.Dataset.new()
dset.append_data(rmdata)

lh = Ncm.Likelihood.new(dset)

mset.param_set_all_ftype(Ncm.ParamType.FREE)
mset.param_set_ftype(7000, 12, Ncm.ParamType.FIXED)
mset.param_set_ftype(7000, 13, Ncm.ParamType.FIXED)
mset.param_set_ftype(7000, 14, Ncm.ParamType.FIXED)


mset.prepare_fparam_map()

fit = Ncm.Fit.factory( Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL )

fit.log_info()

In [ ]:
fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)

In [ ]:
fit.log_info()

In [ ]:
lnR_mean_ext2 = np.array([ext2.get_mean_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])
lnR_std_ext2 = np.array( [ext2.get_std_richness(lnM_mean[i], z_mean[i]) for i in range(len(halos_mean))])

## Comparison I

In [ ]:
#### Ascaso ####

fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

zs = lnR_mean_ascaso
xs = z_mean
ys = lnM_mean

p3 =ax1.scatter(xs, ys, zs, c=zs, cmap='RdYlBu')
ax1.set_xlabel('z')
ax1.set_ylabel('lnM')
#ax1.set_zlabel('mu0')
fig.colorbar(p3, ax=ax1, label='LnR mean')

#data comparison
dc_lnR = [np.mean(j) for j in lnR_binned if len(j)>0]
dc_z = [np.mean(k) for k in z_binned if len(k) > 0]
dc_lnM = [np.mean(l) for l in lnM_binned if len(l) > 0]


ax2.scatter(dc_lnM, dc_lnR, c='k')
p4 = ax2.scatter(ys, zs , c= xs, cmap='RdYlBu')
ax2.set_xlabel('lnM')
ax2.set_ylabel('LnR mean')
fig.colorbar(p4, ax=ax2, label='z')

ax3.scatter(dc_z, dc_lnR, c='k')
p5 = ax3.scatter(xs, zs , c=ys, cmap='RdYlBu') #Ascaso <lnR|M,z> = \mu_0, dados binados
ax3.set_xlabel('z')
ax3.set_ylabel('LnR mean')
fig.colorbar(p5, ax=ax3, label='lnM')

plt.show()


#### Extended (ln(1 + z)) ####

fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

zs = lnR_mean_ext
xs = z_mean
ys = lnM_mean

p3 =ax1.scatter(xs, ys, zs, c=zs, cmap='RdYlBu')
ax1.set_xlabel('z')
ax1.set_ylabel('lnM')
#ax1.set_zlabel('mu0')
fig.colorbar(p3, ax=ax1, label='LnR mean')

#data comparison
dc_lnR = [np.mean(j) for j in lnR_binned if len(j)>0]
dc_z = [np.mean(k) for k in z_binned if len(k) > 0]
dc_lnM = [np.mean(l) for l in lnM_binned if len(l) > 0]

ax2.scatter(dc_lnM, dc_lnR, c='k')
p4 = ax2.scatter(ys, zs , c= xs, cmap='RdYlBu')
ax2.set_xlabel('lnM')
ax2.set_ylabel('LnR mean')
fig.colorbar(p4, ax=ax2, label='z')

ax3.scatter(dc_z, dc_lnR, c='k')
p5 = ax3.scatter(xs, zs , c=ys, cmap='RdYlBu') #Ascaso <lnR|M,z> = \mu_0, dados binados
ax3.set_xlabel('z')
ax3.set_ylabel('LnR mean')
fig.colorbar(p5, ax=ax3, label='lnM')

plt.show()


#### Extended (z) ####

fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

zs = lnR_mean_ext2
xs = z_mean
ys = lnM_mean

p3 =ax1.scatter(xs, ys, zs, c=zs, cmap='RdYlBu')
ax1.set_xlabel('z')
ax1.set_ylabel('lnM')
#ax1.set_zlabel('mu0')
fig.colorbar(p3, ax=ax1, label='LnR mean')

#data comparison
dc_lnR = [np.mean(j) for j in lnR_binned if len(j)>0]
dc_z = [np.mean(k) for k in z_binned if len(k) > 0]
dc_lnM = [np.mean(l) for l in lnM_binned if len(l) > 0]

ax2.scatter(dc_lnM, dc_lnR, c='k')
p4 = ax2.scatter(ys, zs , c= xs, cmap='RdYlBu')
ax2.set_xlabel('lnM')
ax2.set_ylabel('LnR mean')
fig.colorbar(p4, ax=ax2, label='z')

ax3.scatter(dc_z, dc_lnR, c='k')
p5 = ax3.scatter(xs, zs , c=ys, cmap='RdYlBu') #Ascaso <lnR|M,z> = \mu_0, dados binados
ax3.set_xlabel('z')
ax3.set_ylabel('LnR mean')
fig.colorbar(p5, ax=ax3, label='lnM')

plt.show()


## Comparison II

In [ ]:
dt_halos = Table(DC2_halos_m200c[1].data)

# lnM = np.array(np.log(dt_halos["m200c"]))  # ln(mass)
# z = np.array(dt_halos["redshift_true"])  # redshift
lnR = np.array(np.log(dt_halos["richness"]))  # ln(richness)

mean_ext = [ext.get_mean_richness(i, j) for i, j in zip(lnM, z)]
std_ext = [ext.get_std_richness(i, j) for i, j in zip(lnM, z)]

In [ ]:
fig = plt.figure(figsize=(16,8))

# ax1 = fig.add_subplot(1,3,1, projection='3d')
# ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,1,1)

zs = mean_ext
xs = z
ys = lnM


#p5 = ax3.scatter(xs, zs , c=ys, cmap='RdYlBu') #Ascaso <lnR|M,z> = \mu_0, dados binados

z_a = np.linspace(np.min(z), np.max(z), 6)
lnM_a = np.linspace(np.min(lnM), np.max(lnM), 1000)

for j in range(0, 61, 6):
    z_j = np.mean(z_binned[j * 21])
    
    ax3.plot(lnM_a, [ext.get_mean_richness(lnM_i, z_j) for lnM_i in lnM_a])
    t1 = [np.mean(lnM_binned[j * 21 + i ]) for i in range(21)]
    t2 = [np.mean(lnR_binned[j * 21 + i ]) for i in range(21)]
    
    ax3.scatter(t1, t2, s=4.5)
    

#p5 = ax3.scatter(lnM_mean, lnR_mean , c=z_mean, cmap='RdYlBu', s=0.4)
ax3.set_xlabel('lnM')
ax3.set_ylabel('LnR mean')
#fig.colorbar(p5, ax=ax3, label='lnM')

plt.show()


## Comparison III

In [ ]:
fig = plt.figure(figsize=(16,8))

# ax1 = fig.add_subplot(1,3,1, projection='3d')
# ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,1,1)

#p5 = ax3.scatter(xs, zs , c=ys, cmap='RdYlBu') #Ascaso <lnR|M,z> = \mu_0, dados binados

z_a = np.linspace(np.min(z), np.max(z), 4)
lnM_a = np.linspace(np.min(lnM), np.max(lnM), 1000)

for j in range(0, 60, 4):
    z_j = np.mean(z_binned[j * 21])
    
    ax3.plot(lnM_a, [ext.get_std_richness(lnM_i, z_j) for lnM_i in lnM_a])
    t1 = [np.mean(lnM_binned[j * 21 + i ]) for i in range(21)]
    t2 = [np.std(lnR_binned[j * 21 + i ]) for i in range(21)]
    
    ax3.scatter(t1, t2)
    ax3.plot(t1, t2)




#p5 = ax3.scatter(lnM_mean, lnR_mean , c=z_mean, cmap='RdYlBu', s=0.4)
ax3.set_xlabel('lnM')
ax3.set_ylabel('LnR std')
#fig.colorbar(p5, ax=ax3, label='lnM')

plt.show()


In [ ]:
fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

zs = mean_ext
xs = z
ys = lnM

p3 =ax1.scatter(xs, ys, zs, c=zs, cmap='RdYlBu')
ax1.set_xlabel('z')
ax1.set_ylabel('lnM')
#ax1.set_zlabel('mu0')
fig.colorbar(p3, ax=ax1, label='LnR mean')

p4 = ax2.scatter(ys, zs , c= xs, cmap='RdYlBu')
ax2.set_xlabel('lnM')
ax2.set_ylabel('LnR mean')
fig.colorbar(p4, ax=ax2, label='z')

p5 = ax3.scatter(xs, zs , c=ys, cmap='RdYlBu') #Ascaso <lnR|M,z> = \mu_0, dados binados
ax3.set_xlabel('z')
ax3.set_ylabel('LnR mean')
fig.colorbar(p5, ax=ax3, label='lnM')

plt.show()

fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(1,3,1, projection='3d')
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)

zs = lnR

p3 =ax1.scatter(xs, ys, zs, c=zs, cmap='RdYlBu')
ax1.set_xlabel('z')
ax1.set_ylabel('lnM')
#ax1.set_zlabel('mu0')
fig.colorbar(p3, ax=ax1, label='LnR mean')

p4 = ax2.scatter(ys, zs , c= xs, cmap='RdYlBu')
ax2.set_xlabel('lnM')
ax2.set_ylabel('LnR mean')
fig.colorbar(p4, ax=ax2, label='z')

p5 = ax3.scatter(xs, zs , c=ys, cmap='RdYlBu') #Ascaso <lnR|M,z> = \mu_0, dados binados
ax3.set_xlabel('z')
ax3.set_ylabel('LnR mean')
fig.colorbar(p5, ax=ax3, label='lnM')

plt.show()